### データ収集してcsvファイルで保存

In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

import time

In [2]:
!bs4 --version

/bin/bash: bs4: command not found


In [2]:
#１つの建物の情報
all_data=[]
max_page=1
suumo_url='https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

s=time.time()
def get_html(url):
    res=requests.get(url)
    soup=BeautifulSoup(res.content, "html.parser")
    return soup

for page in range(1, max_page+1):
    url=suumo_url.format(page)
    soup=get_html(url)
    #読み込みによるサーバーダウンを防ぐために一時中断(2022/09/30: 追加)
    time.sleep(1)
    items=soup.find_all('div', class_="cassetteitem")
    
    
    for item in items:    
        #各物件に関して、物件情報の共通項目：最寄駅
        stations=item.find_all('div', class_="cassetteitem_detail-text")

        base_data={}
        base_data['物件名']=item.find('div', class_="cassetteitem_content-title").text
        base_data['カテゴリー']=item.find('div', class_="cassetteitem_content-label").text
        base_data['住所']=item.find('li', class_="cassetteitem_detail-col1").text
        base_data['最寄駅0']=stations[0].text
        base_data['最寄駅1']=stations[1].text
        base_data['最寄駅2']=stations[2].text   
        base_data['築年数']=item.find('li', class_="cassetteitem_detail-col3").find_all('div')[0].text
        base_data['構造']=item.find('li', class_="cassetteitem_detail-col3").find_all('div')[1].text


        #各部屋に関して
        tbodys=item.find('table', class_="cassetteitem_other").find_all('tbody')

        for i,tbody in enumerate(tbodys):
            data=base_data.copy()

            data['階数']=tbody.find_all('td')[2].getText().strip()
            data['賃料']=tbody.find('span', class_="cassetteitem_other-emphasis ui-text--bold").getText().strip()
            data['管理費']=tbody.find('span', class_="cassetteitem_price cassetteitem_price--administration").getText().strip()
            data['敷金']=tbody.find('span', class_="cassetteitem_price cassetteitem_price--deposit").getText().strip()
            data['礼金']=tbody.find('span', class_="cassetteitem_price cassetteitem_price--gratuity").getText().strip()
            data['間取り']=tbody.find('span', class_="cassetteitem_madori").getText().strip()
            data['面積']=tbody.find('span', class_="cassetteitem_menseki").getText().strip()

            all_data.append(data)    

df=pd.DataFrame(all_data)
e=time.time()

print('処理時間：', e-s)

処理時間： 141.47361087799072


In [3]:
#収集した内容の確認
df.head(5)       

,物件名,カテゴリー,住所,最寄駅0,最寄駅1,最寄駅2,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積
0,プチ・ボヌール,賃貸アパート,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2
1,Ｃａｓａ荒川 ｐａｒｔ３,賃貸アパート,東京都荒川区荒川１,東京メトロ千代田線/町屋駅 歩14分,ＪＲ常磐線/三河島駅 歩14分,東京メトロ日比谷線/三ノ輪駅 歩14分,築8年,3階建,1階,8.8万円,3000円,-,8.8万円,1SK,31.65m2
2,コンフォリア町屋,賃貸マンション,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,2階,12.1万円,7000円,12.1万円,-,1LDK,39m2
3,コンフォリア町屋,賃貸マンション,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,5階,12.2万円,7000円,12.2万円,-,1LDK,39m2
4,ＭＡＩＳＯＮ．Ｏ．,賃貸マンション,東京都荒川区南千住１,東京メトロ日比谷線/三ノ輪駅 歩8分,ＪＲ常磐線/南千住駅 歩8分,都電荒川線/三ノ輪橋駅 歩5分,新築,6階建,2階,13.1万円,10000円,13.1万円,13.1万円,1LDK,42.32m2


In [4]:
#csvファイルで保存
df.to_csv('sample_data.csv', index=None)

In [5]:
#試しに読み込み
df_sample=pd.read_csv('sample_data.csv')
df_sample.head()

,物件名,カテゴリー,住所,最寄駅0,最寄駅1,最寄駅2,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積
0,プチ・ボヌール,賃貸アパート,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2
1,Ｃａｓａ荒川 ｐａｒｔ３,賃貸アパート,東京都荒川区荒川１,東京メトロ千代田線/町屋駅 歩14分,ＪＲ常磐線/三河島駅 歩14分,東京メトロ日比谷線/三ノ輪駅 歩14分,築8年,3階建,1階,8.8万円,3000円,-,8.8万円,1SK,31.65m2
2,コンフォリア町屋,賃貸マンション,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,2階,12.1万円,7000円,12.1万円,-,1LDK,39m2
3,コンフォリア町屋,賃貸マンション,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,5階,12.2万円,7000円,12.2万円,-,1LDK,39m2
4,ＭＡＩＳＯＮ．Ｏ．,賃貸マンション,東京都荒川区南千住１,東京メトロ日比谷線/三ノ輪駅 歩8分,ＪＲ常磐線/南千住駅 歩8分,都電荒川線/三ノ輪橋駅 歩5分,新築,6階建,2階,13.1万円,10000円,13.1万円,13.1万円,1LDK,42.32m2


In [6]:
df_sample.shape

(4348, 15)